## Limpieza del archivo .xml
Este código sirve para la lectura, limpieza y actualización de archivos xml.
Para usarlo, es necesario que **el archivo esté en la carpeta actual**.

Se deben seguir los siguientes pasos:

- xml = Procesado_xml('nombre-de-tu-archivo.xml')

- xml_limpio = xml.limpieza_xml()

Como resultado, se obtiene el archivo limpio en forma de lista.

En este mismo archivo dejamos las celdas necesarias con este código para su ejecución.


In [1]:
import re
import os
import xml.etree.ElementTree as ET

class Procesado_xml:
    '''Usaremos esta clase para lectura y limpieza de archivo xml.
    Contiene dos métodos uno de lectura (lectura_xml) y otro de limpieza (limpieza_xml).
    En el de limpieza también se incluye el de lectura.
    Return: Una lista del archivo procesado y limpio.'''

    def __init__(self, ruta_archivo_xml):
        self.ruta_archivo_xml = ruta_archivo_xml
    # Métodos: funciones de lectura y de limpieza
    def lectura_xml(self):
        tree = ET.parse(self.ruta_archivo_xml)
        root = tree.getroot()

        xml_list = []
        for child in root:
            xml_dict = {}
            for subchild in child:
                xml_dict[subchild.tag] = subchild.text
            xml_list.append(xml_dict) # Es una lista que contiene diccionarios
        
        return xml_list

    def limpieza_xml (self):
        xml_list = self.lectura_xml()
        for i in xml_list:
            if i['gender'] == '0':
                i['gender'] = 'man'
            elif i['gender'] == '1':
                i['gender'] = 'woman'
            elif i['gender'] == '2':
                i['gender'] = 'nonbinary'
            elif i['gender'] == '3':
                i['gender'] = 'prefer not to say'
            elif i['gender'] == '4':
                i['gender'] = 'prefer to self-describe'
        for i in xml_list:
            i.pop('level_0')
        return  xml_list

    def conversion_xml(self):
        xml_list = self.limpieza_xml()
        xml_final = []
        for i in xml_list:
            tupla = tuple(i.values())
            xml_final.append(tupla)
        return xml_final

In [2]:
xml = Procesado_xml('data_xml.xml')

In [5]:
xml_final = xml.conversion_xml()

In [ ]:
# Mediante este código creamos la tabla data_txt en MySQL
import mysql.connector

cnx = mysql.connector.connect(user='root', password = 'AlumnaAdalab' ,
                            host='127.0.0.1', database ='project1')

mycursor = cnx.cursor()

try:
    mycursor.execute("""CREATE TABLE data_xml (
                            index_xml int not null auto_increment,
                            `time` varchar(255),
                            age varchar (255),
                            gender varchar (255),
                            index_sql int,
                            primary key (index_xml),
                            constraint `fk_data_xml_data_sql` 
                            foreign key (index_sql)
                            references `data_sql` (index_sql)
                            on delete cascade on update cascade
                            );""")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

mycursor.close()
cnx.close()

In [8]:
# Función para la insercción de datos a la tabla datos_txt

def insercion_datos_xml(datos_xml):
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                host='127.0.0.1', database='project1')


    mycursor = cnx.cursor()
    
    sql = """INSERT INTO data_xml (index_xml, `time`, age, gender) VALUES (%s,%s,%s,%s)"""
    val = datos_xml
    
    try: 
        mycursor.executemany(sql, val)
        cnx.commit()
        print(mycursor.rowcount, "registro insertado.")

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)


In [9]:
insercion_datos_xml(xml_final)

25972 registro insertado.
